In [ ]:
import warnings
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numba as nb
import torch
import statsmodels.api as sm
from tqdm import trange, tqdm
from datetime import datetime
from functools import partial
from sklearn.linear_model import LogisticRegression
from scipy.stats import gaussian_kde
import time
import colorama
from colorama import Fore, Style
from sklearn.neural_network import MLPClassifier
from scipy.stats import kstest
from itertools import combinations
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, ShuffleSplit, train_test_split
from sklearn.utils import resample
from scipy.linalg import cholesky, solve_triangular
from scipy.special import logsumexp
from scipy.optimize import minimize
from scipy.integrate import dblquad, simps
from scipy.spatial import KDTree
from scipy import interpolate
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KernelDensity, NearestNeighbors, KNeighborsClassifier
from sklearn.metrics import roc_auc_score, mean_squared_error, r2_score, log_loss
from sklearn.cluster import KMeans
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBClassifier, XGBRegressor
import geopandas as gpd
import seaborn as sns
from scipy.spatial.distance import cdist
from cvxopt import matrix, solvers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from itertools import combinations

solvers.options['disp'] = False
solvers.options['show_progress'] = False

In [ ]:
def kernel_mean_matching(g_train, g_test, kern='lin', B=1.0, eps=None):
    nx = g_train.shape[0]
    nz = g_test.shape[0]
    
    if eps is None:
        eps = max(1e-6, B / np.sqrt(nz))  # Avoid very small values for uniform distributions
    
    if kern == 'lin':
        K = np.dot(g_test, g_test.T)
        kappa = np.sum(np.dot(g_test, g_train.T) * float(nz) / float(nx), axis=1)
    elif kern == 'rbf':
        K = compute_rbf(g_test, g_test, sigma=adjust_sigma(g_test))
        kappa = np.sum(compute_rbf(g_test, g_train, sigma=adjust_sigma(g_test)), axis=1) * float(nz) / float(nx)
    else:
        raise ValueError('Unknown kernel')
    
    K = matrix(K)
    kappa = matrix(kappa)
    
    # Regularization with dynamic epsilon
    G = matrix(np.vstack([np.ones((1, nz)), -np.ones((1, nz)), np.eye(nz), -np.eye(nz)]))
    h = matrix(np.hstack([nz * (1 + eps), nz * (eps - 1), B * np.ones(nz), np.zeros(nz)]))
    
    sol = solvers.qp(K, -kappa, G, h)
    coef = np.array(sol['x']).flatten()
    
    # Clip the coefficients to avoid extreme values
    coef = np.clip(coef, 0, B)
    #print(f"coef:{coef}")
    return coef

def compute_rbf(X, Z, sigma=1.0):
    """ Compute RBF kernel matrix """
    K = np.zeros((X.shape[0], Z.shape[0]), dtype=float)
    for i, vx in enumerate(X):
        K[i, :] = np.exp(-np.sum((vx - Z) ** 2, axis=1) / (2.0 * sigma))
    return K

def adjust_sigma(data):
    """ Dynamically adjust sigma based on the variance of the data """
    pairwise_dists = np.sum((data[:, None] - data[None, :])**2, axis=-1)
    median_dist = np.median(pairwise_dists)
    return median_dist / np.log(len(data))  # Use median distance scaled by the log of the sample size

def KMM_error(err, p_sample, g_sample, hyperparam):
    coef = kernel_mean_matching(p_sample, g_sample, kern='rbf', B=hyperparam)
    return logsumexp(err(g_sample) + np.log(coef)) - np.log(g_sample.shape[0])


In [ ]:
def estimate_lcf(pn_est_df, r=None, dim=None, est_name='LCF'):

    if r is None:
        r = pn_est_df['r'].values

    non_zero_indices = np.where(pn_est_df['pn'] != 0)[0]
    if len(non_zero_indices) == 0:
        lcf_df = pd.DataFrame({'r': r, 'theo': 0, est_name: -1})
        return lcf_df

    first_non_zero_ind = non_zero_indices[0]

    if np.any(np.isnan(pn_est_df['pn'])):
        nan_indices = np.where(np.isnan(pn_est_df['pn']))[0]
        if len(nan_indices) == 0:
            last_ind = len(pn_est_df) - 1
        else:
            last_ind = min(len(pn_est_df) - 1, nan_indices[0] - 1)
    else:
        last_ind = len(pn_est_df) - 1

    pn_est_defined = pn_est_df.iloc[first_non_zero_ind:last_ind + 1]

    if dim is None:
        dim = int(np.sqrt(len(pn_est_df)))
    
    dim = max(2, dim)  # Minimum 2 basis functions to ensure a valid spline degree
    spline_degree = min(dim - 1, 5)  # Ensure degree is at most 5
    
    spline = interpolate.UnivariateSpline(pn_est_defined['r'], pn_est_defined['pn'], s=0, k=spline_degree, ext='raise')
    spline_derivative = spline.derivative()

    r_li = np.where(r >= pn_est_defined['r'].iloc[0])[0]
    if len(r_li) > 0:
        r_li = r_li[0]
    else:
        r_li = 0

    r_hi = np.where(r > pn_est_defined['r'].iloc[-1])[0]
    if len(r_hi) > 0:
        r_hi = r_hi[0] - 1
    else:
        r_hi = len(r) - 1

    if r_li <= r_hi:
        r_def = r[r_li:r_hi+1]
        pn = spline(r_def)
        pn_deriv = spline_derivative(r_def)
        pn_deriv = np.maximum(pn_deriv, 0)

        lcf = np.where((pn > 0) & (pn_deriv >= 0), compute_lcf(r_def, pn, pn_deriv), -1)

        num_ll_pad = r_li
        num_na_pad = len(r) - r_hi - 1
    else:
        lcf = np.full(len(r), np.nan)
        num_ll_pad = len(r)
        num_na_pad = 0

    lcf = np.concatenate([np.full(num_ll_pad, -1), lcf, np.full(num_na_pad, np.nan)])
    lcf_df = pd.DataFrame({'r': r, 'theo': 0})
    lcf_df[est_name] = lcf

    return lcf_df

def compute_lcf(r, pn, pn_deriv, lcf_lims=(-1, 1)):
    if len(lcf_lims) != 2 or not all(isinstance(i, (int, float)) for i in lcf_lims):
        raise ValueError("lcf_lims should be a tuple of two numeric values.")

    if lcf_lims[0] > lcf_lims[1]:
        raise ValueError("The first limit of lcf_lims must be less than the second.")

    if not all(isinstance(arr, np.ndarray) for arr in [r, pn, pn_deriv]):
        raise ValueError("r, pn, and pn_deriv must be numpy arrays.")

    if len(r) != len(pn) or len(pn) != len(pn_deriv):
        raise ValueError("r, pn, and pn_deriv must be of the same length.")

    scale = lcf_lims[1] - lcf_lims[0]
    shift = lcf_lims[0]
    lcf = np.exp(-np.log(2) / 2 * r * pn_deriv / pn) * scale + shift
    return lcf
    
def estimate_point_counts(random_points, r_values):
    tree = KDTree(random_points)
    num_points = len(random_points)

    pn_list = []

    for r in r_values:
        counts = [len(tree.query_ball_point(point, r)) - 1 for point in random_points]
        average_count = np.mean(counts)
        pn_list.append([r, average_count])

    pn_est_df = pd.DataFrame(pn_list, columns=['r', 'pn'])
    
    return pn_est_df


def plot_lcf(x, ylim=(-1, 1), title="LCF Plot", AUC = None):
    plt.figure()
    plt.plot(x['r'], x['LCF'], label=f'LCF with AUC = {AUC}', color='blue')
    plt.axhline(y=0, color='grey', linestyle='--')
    plt.title(title)
    plt.xlabel('r')
    plt.ylabel('LCF')
    plt.legend()
    plt.show()
    return True 
    
def compute_normalized_auc(lcf_df, r_values, rmin=None, rmax=None):
    if rmin is None:
        rmin = np.min(r_values)
    if rmax is None:
        rmax = np.max(r_values)

    mask = (r_values >= rmin) & (r_values <= rmax)
    r_filtered = r_values[mask]
    lcf_filtered = lcf_df['LCF'].values[mask]

    delta_r = rmax - rmin

    area_under_curve = np.trapz(lcf_filtered, r_filtered)

    normalized_auc = area_under_curve/ delta_r

    return normalized_auc

def LCF_AUC_from_sample(sample, name, subname, r_values, rmin, rmax, flag = False):
    est_df = estimate_point_counts(sample, r_values)
    
    lcf_df = estimate_lcf(est_df, r=r_values)

    AUC = compute_normalized_auc(lcf_df, r_values, rmin, rmax)

    if flag:
        plot_lcf(
                lcf_df,
                title=f"{name} -- {subname}",
                AUC=AUC,
                )
    return AUC

# Cells dataset

In [ ]:
base_dir = '..//datasets/cells'
cells_list = ["B cell", "Myeloid cell"]
use_pca = True  
pca_num = 2      
df_dict = {}
DOWNSAMPLE_THRESHOLD = 2000 

for cell_combination in combinations(cells_list, 2):
    combo_key = "-".join(cell_combination)
    df_dict.setdefault(combo_key, [])

    for tma_folder in ['TMA9_1A_Scan1', 'TMA9_1B_Scan1', 'TMA9_2A_Scan1', 'TMA9_2B_Scan1']:
        tma_path = os.path.join(base_dir, tma_folder)

        for folder in os.listdir(tma_path):
            if folder.startswith("T0"):
                folder_path = os.path.join(tma_path, folder)
                subfolder_path = os.path.join(folder_path, "1")

                if os.path.isdir(subfolder_path):
                    prediction_file = os.path.join(subfolder_path, 'prediction.txt')

                    if os.path.isfile(prediction_file):
                        try:
                            df = pd.read_csv(prediction_file, header=None, sep='\t')
                            df.columns = ['Latitude', 'Longitude', 'Feature 1', 'Feature 2',
                                          'Feature 3', 'Feature 4', 'Feature 5', 'Metrics']
                        except pd.errors.EmptyDataError:
                            print(f"Warning: Empty or invalid file: {prediction_file}")
                            continue
                        except Exception as e:
                            print(f"Warning: Error reading file {prediction_file}: {e}")
                            continue


                        df = df[df['Metrics'].isin(cell_combination)]
                        label_mapping = {cell: idx for idx, cell in enumerate(cell_combination)}
                        df['Metrics'] = df['Metrics'].map(label_mapping)

                        df.dropna(subset=['Metrics'], inplace=True)
                        df['Metrics'] = df['Metrics'].astype(int)


                        if len(df['Metrics'].unique()) < 2 or len(df) < 1000: 
                            continue

                        X = df.drop(columns=['Metrics'])
                        y = df['Metrics']

                        X_train, X_test, y_train, y_test = train_test_split(
                            X, y, test_size=0.5, stratify=y, random_state=43
                        )

                        if len(X_train) > DOWNSAMPLE_THRESHOLD:
                            X_train, y_train = resample(
                                X_train, y_train,
                                n_samples=DOWNSAMPLE_THRESHOLD,
                                replace=False,  
                                stratify=y_train,
                                random_state=42
                            )

                        if len(X_test) > DOWNSAMPLE_THRESHOLD:
                            X_test, y_test = resample(
                                X_test, y_test,
                                n_samples=DOWNSAMPLE_THRESHOLD,
                                replace=False,  
                                stratify=y_test,
                                random_state=42
                            )
                        
                        scaler = StandardScaler()

                        if use_pca:
                            if X_train.shape[0] < pca_num or X_train.shape[1] < pca_num:
                                print(f"Skipping {folder} for {combo_key} due to insufficient samples/features for PCA: {X_train.shape}")
                                continue
                            pca = PCA(n_components=pca_num)
                            X_train_transformed = pca.fit_transform(X_train)
                            X_test_transformed = pca.transform(X_test)
                            columns = [f'PC{i+1}' for i in range(pca_num)]
                        else:
                            X_train_transformed = X_train.values
                            X_test_transformed = X_test.values
                            columns = X_train.columns.tolist()

                        X_train_scaled = scaler.fit_transform(X_train_transformed)
                        X_test_scaled = scaler.transform(X_test_transformed)

                        train_df = pd.DataFrame(X_train_scaled, columns=columns)
                        train_df['Metrics'] = y_train.reset_index(drop=True)

                        test_df = pd.DataFrame(X_test_scaled, columns=columns)
                        test_df['Metrics'] = y_test.reset_index(drop=True)

                        df_dict[combo_key].append((train_df, test_df, folder))

In [ ]:
samples_dict = {}

for key in df_dict:
    samples_dict[key] = {'auc_values': [], 'df_list': []}  
    
    for train_df, test_df, subname in df_dict[key]:
        combined_df = pd.concat([train_df, test_df], axis=0).reset_index(drop=True)

        #edit radius limits for different dimensions for pattern detection
        rmax = 0.188 
        rmin = rmax / 10
        r_values = np.arange(0, rmax, rmax / 50)
        
        sample = np.asarray(combined_df.drop(columns=['Metrics']).values.tolist())
        
        AUC = LCF_AUC_from_sample(sample, key, subname, r_values, rmin, rmax, flag=False)
        samples_dict[key]['auc_values'].append(AUC)  
        samples_dict[key]['df_list'].append((train_df, test_df, subname, AUC))  
        
        print(f"AUC for {key}, {subname}: {AUC}")

In [ ]:
AUC_p_thr = 0.1
AUC_g_thr = 0.15

for key in samples_dict:
    samples_dict[key]['df_list_p'] = []
    samples_dict[key]['df_list_g'] = []
    
    for train_df, test_df, subname, AUC in samples_dict[key]['df_list']:
        if np.abs(AUC) < np.abs(AUC_p_thr):
            samples_dict[key]['df_list_p'].append((train_df, test_df, subname, AUC))
        elif np.abs(AUC) > AUC_g_thr :
            samples_dict[key]['df_list_g'].append((train_df, test_df, subname, AUC))

for key in samples_dict:
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))        
    sns.boxplot(data=[samples_dict[key]['auc_values']], ax=ax)
    ax.set_xticklabels(['AUC'])
    ax.set_title(f"Boxplot of AUC_p and AUC_g for {key}")
    ax.set_ylabel("AUC values")
    ax.axhline(AUC_p_thr, color='red', linestyle='--', label='AUC_p_thr')
    ax.axhline(AUC_g_thr, color='blue', linestyle='--', label='AUC_g_thr')
    ax.legend()
    plt.tight_layout()
    plt.show()

for key in samples_dict:
    print(f"Combination: {key}")
    print(f"Number of p-type samples: {len(samples_dict[key]['df_list_p'])}")
    print(f"Number of g-type samples: {len(samples_dict[key]['df_list_g'])}\n")

In [ ]:
AUC_p_thr = 0.1
AUC_g_thr = 0.15

base_plot_dir = './pca'
os.makedirs(base_plot_dir, exist_ok=True)

for key in samples_dict: 
    samples_dict[key]['df_list_p'] = []
    samples_dict[key]['df_list_g'] = []
    
    if 'df_list' in samples_dict[key] and samples_dict[key]['df_list']:
        for train_df, test_df, subname, auc_score_val in samples_dict[key]['df_list']:
            if not isinstance(auc_score_val, (int, float)):
                print(f"Warning: Invalid AUC value ({auc_score_val}) for {subname} in {key}. Skipping p/g classification.")
                continue

            if np.abs(auc_score_val) < np.abs(AUC_p_thr):
                samples_dict[key]['df_list_p'].append((train_df, test_df, subname, auc_score_val))
            elif np.abs(auc_score_val) > AUC_g_thr:
                samples_dict[key]['df_list_g'].append((train_df, test_df, subname, auc_score_val))
    else:
        print(f"Info: No 'df_list' or empty 'df_list' for key {key} to classify into p/g types.")

# if use_pca and pca_num == 2:
#     print(f"\nGenerating PCA plots and CSVs (use_pca={use_pca}, pca_num={pca_num})...")
#     for type_label, type_list_key in [('p', 'df_list_p'), ('g', 'df_list_g')]:
#         type_specific_dir = os.path.join(base_plot_dir, type_label)
#         os.makedirs(type_specific_dir, exist_ok=True)

#         for combo_key_plot in samples_dict: # combo_key_plot is the cell combination string
#             combo_specific_dir = os.path.join(type_specific_dir, combo_key_plot)
#             os.makedirs(combo_specific_dir, exist_ok=True)

#             if samples_dict[combo_key_plot] and type_list_key in samples_dict[combo_key_plot] and samples_dict[combo_key_plot][type_list_key]:
#                 for train_df_plot, test_df_plot, subname_plot, auc_val_plot in samples_dict[combo_key_plot][type_list_key]:
#                     if train_df_plot.empty and test_df_plot.empty:
#                         continue
                    
#                     plot_df = pd.concat([train_df_plot, test_df_plot], ignore_index=True)

#                     csv_filename = f"{subname_plot}.csv"
#                     csv_path = os.path.join(combo_specific_dir, csv_filename)
#                     try:
#                         plot_df[['PC1', 'PC2', 'Metrics']].to_csv(csv_path, index=False)
#                     except Exception as e:
#                         print(f"Error saving CSV {csv_path}: {e}")
#                         continue


#                     plt.figure(figsize=(10, 8))
#                     scatter = plt.scatter(plot_df['PC1'], plot_df['PC2'], c=plot_df['Metrics'], cmap='coolwarm', alpha=0.6, edgecolors='k', linewidth=0.5)
#                     plt.xlabel('Principal Component 1')
#                     plt.ylabel('Principal Component 2')
#                     plt.title(f'PCA ({type_label}-type) - {combo_key_plot} - {subname_plot}\nAUC: {auc_val_plot:.3f}', fontsize=10)
                    
#                     handles, labels = scatter.legend_elements(prop="colors", alpha=0.6)
#                     legend_labels = [f'Class {label.split("{")[-1].split("}")[0]}' for label in labels]

#                     plt.legend(handles, legend_labels, title="Classes")
#                     plt.grid(True, linestyle='--', alpha=0.7)
                    
#                     png_filename = f"{subname_plot}.png"
#                     png_path = os.path.join(combo_specific_dir, png_filename)
#                     try:
#                         plt.savefig(png_path)
#                     except Exception as e:
#                         print(f"Error saving PNG {png_path}: {e}")
#                     plt.close() 
# else:
#     if not use_pca:
#         print("\nPCA is disabled (use_pca=False). Skipping PCA plot generation.")
#     elif pca_num != 2:
#         print(f"\nPCA is enabled but pca_num is {pca_num} (not 2). Skipping 2D PCA plot generation.")

print("\nGenerating AUC boxplots...")
for key in samples_dict: 
    if samples_dict[key] and 'auc_values' in samples_dict[key] and samples_dict[key]['auc_values']:
        fig, ax = plt.subplots(1, 1, figsize=(8, 6))        
        sns.boxplot(y=samples_dict[key]['auc_values'], ax=ax, showfliers=True, color="lightblue") # Using y for vertical boxplot
        ax.set_ylabel("AUC values")
        ax.set_title(f"Boxplot of AUC values for {key}")
        ax.axhline(AUC_p_thr, color='red', linestyle='--', label=f'AUC_p_thr ({AUC_p_thr:.2f})')
        ax.axhline(AUC_g_thr, color='blue', linestyle='--', label=f'AUC_g_thr ({AUC_g_thr:.2f})')
        ax.set_ylim(-0.05, 1.05) 
        ax.legend()
        plt.tight_layout()
        plt.show()
    else:
        print(f"Info: No 'auc_values' or empty list for key {key}. Skipping boxplot.")

print("\nCounts of p-type and g-type samples:")
for key in samples_dict: 
    num_p_type = 0
    if samples_dict[key] and 'df_list_p' in samples_dict[key]:
        num_p_type = len(samples_dict[key]['df_list_p'])
    
    num_g_type = 0
    if samples_dict[key] and 'df_list_g' in samples_dict[key]:
        num_g_type = len(samples_dict[key]['df_list_g'])

    total_samples = 0
    if samples_dict[key] and 'df_list' in samples_dict[key]:
        total_samples = len(samples_dict[key]['df_list'])

    print(f"Combination: {key}")
    print(f"  Total processed samples: {total_samples}")
    print(f"  Number of p-type samples (AUC < {AUC_p_thr:.2f}): {num_p_type}")
    print(f"  Number of g-type samples (AUC > {AUC_g_thr:.2f}): {num_g_type}")
    print(f"  Number of samples not classified as p or g: {total_samples - num_p_type - num_g_type}\n")

In [ ]:
colorama.init(autoreset=True)

def custom_log_loss(y_true, y_pred_prob):
    epsilon = 1e-15  
    y_pred_prob = np.clip(y_pred_prob, epsilon, 1 - epsilon)
    log_loss_value = -np.mean(y_true * np.log(y_pred_prob) + (1 - y_true) * np.log(1 - y_pred_prob))
    
    return log_loss_value

def print_header(text, color=Fore.CYAN):
    print(f"\n{color}{Style.BRIGHT}{'=' * 60}")
    print(f"{color}{Style.BRIGHT} {text}")
    print(f"{color}{Style.BRIGHT}{'=' * 60}{Style.RESET_ALL}\n")

def print_subheader(text, color=Fore.YELLOW):
    print(f"\n{color}{Style.BRIGHT} {text}")
    print(f"{color}{Style.BRIGHT}{'-' * 40}{Style.RESET_ALL}")

def print_result(label, value, color=Fore.GREEN):
    print(f"{color}{label}: {Style.RESET_ALL}{value:.3f}")

def print_section_separator():
    print(f"{Fore.BLUE}{'-' * 60}{Style.RESET_ALL}")

def evaluate_model(model_name, model, X_train, y_train, X_test, y_test, p_features_train, p_features_test, df_p_test):
    start_time = time.time()
    
    print_subheader(f"Training {model_name}")
    model.fit(X_train, y_train)
    
    y_pred_prob = model.predict_proba(X_test)
    auc_test = roc_auc_score(y_test, y_pred_prob[:, 1])
    print_result("ROC AUC", auc_test)
    
    if auc_test <= 0.7:
        print(f"{Fore.RED}Too low ROC AUC, skipping this model{Style.RESET_ALL}")
        return None, None
    
    training_time = time.time() - start_time
    print(f"Training completed in {training_time:.2f} seconds")
    
    kde_scott = gaussian_kde(X_train.T, bw_method='scott')
    bw_scott = kde_scott.factor
    
    kde_scott_p = gaussian_kde(p_features_train.T, bw_method='scott')
    bw_scott_p = kde_scott_p.factor

    print_result("Bandwidth (source)", bw_scott)
    print_result("Bandwidth (target)", bw_scott_p)
    
    coef = kernel_mean_matching(p_features_test, X_test, kern='rbf', B=1000)
    
    log_loss_list_g = []
    log_loss_list_p = []
    
    for label, prob in zip(y_test, y_pred_prob):
        log_loss_list_g.append(custom_log_loss(label, prob[1]))
        
    for label, prob in zip(df_p_test['Metrics'], model.predict_proba(df_p_test.drop(columns=['Metrics']))):
        log_loss_list_p.append(custom_log_loss(label, prob[1]))
    
    mce_p = log_loss_list_p                        
    mce_g = log_loss_list_g
    
    is_scott = [error * kde_scott_p(point_p) / kde_scott(point_g) 
                for error, point_g, point_p in zip(log_loss_list_g, X_test, p_features_test)]
    
    kmm = [error * weight for error, weight in zip(log_loss_list_g, coef)]
    
    X_combined = np.vstack([X_train, p_features_train])
    y_combined = np.concatenate([np.ones(len(X_train)), np.zeros(len(p_features_train))])
    
    try:
        domain_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)
        domain_clf.fit(X_combined, y_combined)
    
        source_label_in_y_combined = 1
        target_label_in_y_combined = 0
    
        idx_source_proba = -1
        if hasattr(domain_clf, 'classes_'):
            for i, class_val in enumerate(domain_clf.classes_):
                if class_val == source_label_in_y_combined:
                    idx_source_proba = i
                    break
            if idx_source_proba == -1:
                 raise ValueError(f"Source label {source_label_in_y_combined} not found in domain_clf.classes_ ({domain_clf.classes_}).")
        else: 
            raise AttributeError("domain_clf does not have 'classes_' attribute after fitting.")
    
        domain_probs_p_source_given_x = domain_clf.predict_proba(X_test)[:, idx_source_proba]
    
        n_source_domain_train = np.sum(y_combined == source_label_in_y_combined)
        n_target_domain_train = np.sum(y_combined == target_label_in_y_combined)
    
        if n_target_domain_train == 0:
            print(f"{Fore.YELLOW}Warning: n_target_domain_train is 0 for domain classifier. Weights might be ill-defined. Setting prior_ratio_correction to 1.0.{Style.RESET_ALL}")
            prior_ratio_correction = 1.0
        elif n_source_domain_train == 0:
            print(f"{Fore.YELLOW}Warning: n_source_domain_train is 0 for domain classifier. Weights might be ill-defined (all zero). Setting prior_ratio_correction to 0.0.{Style.RESET_ALL}")
            prior_ratio_correction = 0.0 # Results in zero weights if target_likeness > 0
        else:
            prior_ratio_correction = n_source_domain_train / n_target_domain_train
        
        prob_target_likeness_given_x = 1.0 - domain_probs_p_source_given_x
        
        raw_density_ratios = prob_target_likeness_given_x / (domain_probs_p_source_given_x + 1e-10)
        
        classifier_weights = raw_density_ratios * prior_ratio_correction
        classifier_weights = np.clip(classifier_weights, 1e-7, 1000)
        
        if len(log_loss_list_g) != len(classifier_weights):
            raise ValueError(f"Mismatch in length between log_loss_list_g ({len(log_loss_list_g)}) and classifier_weights ({len(classifier_weights)}). Ensure X_test corresponds to log_loss_list_g.")
    
        classifier = [error * weight for error, weight in zip(log_loss_list_g, classifier_weights)]
        classifier_mean = np.mean(classifier)
    
    except Exception as e:
        print(f"{Fore.RED}Classifier error: {e}{Style.RESET_ALL}")
        if 'log_loss_list_g' in locals() or 'log_loss_list_g' in globals():
            try:
                num_losses = len(log_loss_list_g)
                classifier = [np.nan] * num_losses
            except NameError:
                classifier = [np.nan]
                print(f"{Fore.YELLOW}Warning: log_loss_list_g not defined when Classifier error occurred.{Style.RESET_ALL}")
        else:
            classifier = [np.nan]
        classifier_mean = np.nan
    
    mce_p_mean, mce_g_mean = np.mean(mce_p), np.mean(mce_g)
    is_scott_mean, kmm_mean = np.mean(is_scott), np.mean(kmm)
    
    print_subheader("Mean Error Values")
    print_result("MCE_p (Target)", mce_p_mean)
    print_result("MCE_g (Source)", mce_g_mean)
    print_result("ISE", is_scott_mean)
    print_result("KMM", kmm_mean)
    print_result("Classifier", classifier_mean)
    
    results = {
        "MCE_p": mce_p_mean,
        "MCE_g": mce_g_mean,
        "ISE": is_scott_mean,
        "KMM": kmm_mean,
        "Classifier": classifier_mean
    }
    
    return results, auc_test

models = {
    "GradientBoosting": GradientBoostingClassifier(n_estimators=3, learning_rate=10, max_depth=1, subsample=0.1, max_features=0.1),
    
    "LogisticRegression": LogisticRegression(
        C=1,              
        penalty='l1',           
        max_iter=1,             
        solver='liblinear',     
    ),

    "RandomForest": RandomForestClassifier(
        n_estimators=1,         
        max_depth=1,            
        min_samples_leaf=1,     
        min_samples_split=0.5,  
        class_weight='balanced' 
    ),
    
    "NeuralNetwork": MLPClassifier(
        hidden_layer_sizes=(8, 4),  
        activation='relu',           
        solver='adam',               
        alpha=0.01,                 
        batch_size='auto',           
        learning_rate='constant',    
        learning_rate_init=10,    
        max_iter=5,                 
        early_stopping=True,         
        validation_fraction=0.1,     
        n_iter_no_change=100        
    )
}

all_results = {model_name: {'MCE_g': [], 'MCE_p': [], 'ISE': [], 'KMM': [], 'Classifier': [], 'ROC_AUC': []} 
               for model_name in models.keys()}

for key in samples_dict:
    print_header(f"Dataset: {key}")
    
    dataset_results = {model_name: {'MCE_g': [], 'MCE_p': [], 'ISE': [], 'KMM': [], 'Classifier': [], 'ROC_AUC': []} 
                    for model_name in models.keys()}
    
    for df_p_train, df_p_test, subname_p, AUC_p in samples_dict[key]['df_list_p']:
        p_features_train = df_p_train.drop(columns=['Metrics']).values
        p_features_test = df_p_test.drop(columns=['Metrics']).values
        p_features = np.vstack((p_features_train, p_features_test))
        
        for df_g_train, df_g_test, subname_g, AUC_g in samples_dict[key]['df_list_g']: 
            print_subheader(f"Source: {subname_g}, Target: {subname_p}")
            print(f"Features - Source: {len(p_features)}, Target: {len(p_features)}")
            print_result("AUC (source)", AUC_g)
            print_result("AUC (target)", AUC_p)
            
            X_train = df_g_train.drop(columns=['Metrics']).values
            X_test = df_g_test.drop(columns=['Metrics']).values
            y_train, y_test = df_g_train['Metrics'], df_g_test['Metrics']  
            
            for model_name, model in models.items():
                print_subheader(f"Model: {model_name}")
                
                model_results, auc_score = evaluate_model(
                    model_name, model, X_train, y_train, X_test, y_test, 
                    p_features_train, p_features_test, df_p_test
                )
                
                if model_results:
                    for method, value in model_results.items():
                        dataset_results[model_name][method].append(value)
                        all_results[model_name][method].append(value)
                    
                    dataset_results[model_name]['ROC_AUC'].append(auc_score)
                    all_results[model_name]['ROC_AUC'].append(auc_score)
                
                print_section_separator()
    
    print_header(f"Summary for dataset: {key}")
    
    print_subheader("Average ROC AUC Scores")
    for model_name in models.keys():
        roc_auc_values = dataset_results[model_name]['ROC_AUC']
        if roc_auc_values:
            avg_roc_auc = np.mean(roc_auc_values)
            print_result(f"{model_name}", avg_roc_auc)
    
    print_section_separator()
    
    for model_name in models.keys():
        print_subheader(f"Model: {model_name}")
        mean_metrics_results = {'MCE_g': None, 'ISE': None, 'KMM': None, 'Classifier': None}
        
        for method, values in dataset_results[model_name].items():
            if method != "MCE_p" and method != "ROC_AUC":
                actual = np.array(dataset_results[model_name]["MCE_p"])
                predicted = np.array(values)
                
                if len(actual) > 0:
                    epsilon = 1e-10
                    actual_adj = np.maximum(actual, epsilon)
                    mape = np.mean(np.abs((actual - predicted) / actual_adj)) * 100
                    rmse = np.sqrt(np.mean((actual - predicted) ** 2))
                    rmspe = np.sqrt(np.mean(((actual - predicted) / actual_adj) ** 2)) * 100
                    
                    mean_metrics_results[method] = {
                        'MAPE': mape,
                        'RMSE': rmse,
                        'RMSPE': rmspe
                    }
        
        for metric_name in ['MAPE', 'RMSE', 'RMSPE']:
            print_subheader(f"{metric_name}")
            for method in ['MCE_g', 'ISE', 'KMM', 'Classifier']:
                if mean_metrics_results[method]:
                    print_result(f"{method}", mean_metrics_results[method][metric_name])
    
    print_section_separator()

In [ ]:
def print_final_header(text, color=Fore.CYAN):
    print(f"\n{color}{Style.BRIGHT}{'=' * 80}")
    print(f"{color}{Style.BRIGHT}  {text}")
    print(f"{color}{Style.BRIGHT}{'=' * 80}{Style.RESET_ALL}\n")

def print_final_dataset_header(text, color=Fore.YELLOW):
    print(f"\n{color}{Style.BRIGHT}Dataset: {text}{Style.RESET_ALL}")

def print_final_result(label, value, color=Fore.WHITE, precision=3):
    if isinstance(value, (int, float, np.floating)):
        print(f"  {color}{label}: {value:.{precision}f}{Style.RESET_ALL}")
    else:
        print(f"  {color}{label}: {value}{Style.RESET_ALL}")

def custom_log_loss(y_true, y_pred_prob):
    epsilon = 1e-15
    y_pred_prob = np.clip(y_pred_prob, epsilon, 1 - epsilon)
    log_losses = - (y_true * np.log(y_pred_prob) + (1 - y_true) * np.log(1 - y_pred_prob))
    return log_losses

def generate_models(n_models=2000):
    models = []
    for i in range(n_models):
        if i < n_models // 2:
            model = GradientBoostingClassifier(
                random_state=i,
                n_estimators=np.random.randint(5, 20),
                max_depth=np.random.randint(1, 4),
                learning_rate=np.random.uniform(0.01, 0.3),
                subsample=np.random.uniform(0.3, 0.8)
            )
        else:
            model = RandomForestClassifier(
                random_state=i,
                n_estimators=np.random.randint(5, 20),
                max_depth=np.random.randint(1, 4),
                max_features=np.random.uniform(0.3, 0.8),
                min_samples_split=np.random.randint(5, 20)
            )
        models.append(model)
    return models

K = 5
n_models = 2000

final_results = {}

if 'samples_dict_filtered' in locals() or 'samples_dict_filtered' in globals():
    for key in samples_dict_filtered:

        if len(samples_dict_filtered[key]['df_list_p']) == 0 or len(samples_dict_filtered[key]['df_list_g']) == 0:
            continue

        tuple_p = samples_dict_filtered[key]['df_list_p'][0]
        tuple_g = samples_dict_filtered[key]['df_list_g'][0]

        df_p_train, df_p_test, years_p, AUC_p_original = tuple_p[0],tuple_p[1],tuple_p[2],tuple_p[3]
        df_g_train, df_g_test, years_g, AUC_g_original = tuple_g[0],tuple_g[1],tuple_g[2],tuple_g[3]

        X_p_train = df_p_train.drop(columns=['Metrics'])
        y_p_train = df_p_train['Metrics']
        X_p_test = df_p_test.drop(columns=['Metrics'])
        y_p_test = df_p_test['Metrics']

        X_g_train = df_g_train.drop(columns=['Metrics'])
        y_g_train = df_g_train['Metrics']
        X_g_test = df_g_test.drop(columns=['Metrics'])
        y_g_test = df_g_test['Metrics']

        if X_g_train.shape[1] != X_p_train.shape[1] or X_g_test.shape[1] != X_p_test.shape[1] or X_g_train.shape[1] != X_g_test.shape[1]:
             continue

        n_features = X_g_train.shape[1]

        if len(X_g_train) < 2 or len(X_p_train) < 2 or len(X_g_test) < 2 or len(X_p_test) < 2:
             continue

        models = generate_models(n_models)

        model_info = []

        kde_scott_g = None
        try:
            if len(X_g_train) >= 2 and X_g_train.shape[1] > 0:
                 kde_scott_g = gaussian_kde(X_g_train.values.T, bw_method='scott')
        except np.linalg.LinAlgError:
             kde_scott_g = None
        except ValueError:
             kde_scott_g = None

        kde_scott_p = None
        try:
            if len(X_p_train) >= 2 and X_p_train.shape[1] > 0:
                 kde_scott_p = gaussian_kde(X_p_train.values.T, bw_method='scott')
        except np.linalg.LinAlgError:
             kde_scott_p = None
        except ValueError:
             kde_scott_p = None

        X_combined_train = np.vstack([X_g_train.values, X_p_train.values])
        y_combined_train = np.concatenate([np.ones(len(X_g_train)), np.zeros(len(X_p_train))])

        domain_clf = None
        classifier_weights_per_sample = None
        idx_source_proba = -1

        try:
            if len(np.unique(y_combined_train)) > 1:
                 domain_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
                 domain_clf.fit(X_combined_train, y_combined_train)

                 source_label_in_y_combined = 1
                 if hasattr(domain_clf, 'classes_'):
                     class_list = list(domain_clf.classes_)
                     if source_label_in_y_combined in class_list:
                         idx_source_proba = class_list.index(source_label_in_y_combined)
                     else:
                         raise ValueError(f"Source label {source_label_in_y_combined} not found in domain_clf.classes_ ({domain_clf.classes_}).")
                 else:
                      raise AttributeError("domain_clf does not have 'classes_' attribute after fitting.")

                 n_source_domain_train = np.sum(y_combined_train == source_label_in_y_combined)
                 n_target_domain_train = np.sum(y_combined_train == 0)

                 if n_target_domain_train > 0 and n_source_domain_train > 0:
                      prior_ratio_correction = n_source_domain_train / n_target_domain_train
                 elif n_target_domain_train == 0:
                      prior_ratio_correction = 1.0
                 else:
                      prior_ratio_correction = 0.0

                 domain_probs_p_source_given_x = domain_clf.predict_proba(X_g_test.values)[:, idx_source_proba]
                 prob_target_likeness_given_x = 1.0 - domain_probs_p_source_given_x
                 raw_density_ratios = prob_target_likeness_given_x / (domain_probs_p_source_given_x + 1e-10)
                 classifier_weights_per_sample = np.clip(raw_density_ratios * prior_ratio_correction, 1e-7, 1000)

            else:
                 domain_clf = None
                 classifier_weights_per_sample = None

        except Exception:
            domain_clf = None
            classifier_weights_per_sample = None

        for i, model in enumerate(models):

            try:
                model.fit(X_g_train, y_g_train)

                y_pred_prob_g_test = model.predict_proba(X_g_test)[:, 1] if len(X_g_test) > 0 else np.array([])
                y_pred_prob_p_test = model.predict_proba(X_p_test)[:, 1] if len(X_p_test) > 0 else np.array([])

                auc_g = np.nan
                if len(np.unique(y_g_test)) > 1 and len(y_g_test) > 0:
                     try:
                         auc_g = roc_auc_score(y_g_test, y_pred_prob_g_test)
                     except Exception:
                         pass

                auc_p = np.nan
                if len(np.unique(y_p_test)) > 1 and len(y_p_test) > 0:
                    try:
                        auc_p = roc_auc_score(y_p_test, y_pred_prob_p_test)
                    except Exception:
                         pass

                log_loss_list_g = custom_log_loss(y_g_test, y_pred_prob_g_test) if len(y_g_test) > 0 else np.array([])
                log_loss_list_p = custom_log_loss(y_p_test, y_pred_prob_p_test) if len(y_p_test) > 0 else np.array([])

                mce_p_mean = np.mean(log_loss_list_p) if len(log_loss_list_p) > 0 else np.nan
                mce_g_mean = np.mean(log_loss_list_g) if len(log_loss_list_g) > 0 else np.nan

                ise_mean = np.nan
                if kde_scott_g is not None and kde_scott_p is not None and len(X_g_test) > 0:
                    try:
                         source_densities = kde_scott_g(X_g_test.values.T)
                         target_densities = kde_scott_p(X_g_test.values.T)
                         ise_weights = target_densities / (source_densities + 1e-10)
                         ise_weights = np.clip(ise_weights, 1e-7, 1000)

                         if len(log_loss_list_g) == len(ise_weights) and len(log_loss_list_g) > 0:
                             ise_mean = np.mean(log_loss_list_g * ise_weights)
                         else:
                              ise_mean = np.nan

                    except Exception:
                        ise_mean = np.nan
                elif len(X_g_test) == 0:
                     pass

                classifier_mean = np.nan
                if classifier_weights_per_sample is not None and len(log_loss_list_g) == len(classifier_weights_per_sample) and len(log_loss_list_g) > 0:
                    try:
                        classifier_mean = np.mean(log_loss_list_g * classifier_weights_per_sample)
                    except Exception:
                        classifier_mean = np.nan
                else:
                     classifier_mean = np.nan

                model_info.append({
                    'model_id': i,
                    'model_type': type(model).__name__,
                    'auc_g': auc_g,
                    'auc_p': auc_p,
                    'mce_p': mce_p_mean,
                    'mce_g': mce_g_mean,
                    'ise': ise_mean,
                    'classifier': classifier_mean,
                    'log_loss_list_g': log_loss_list_g
                })

            except Exception:
                model_info.append({
                    'model_id': i,
                    'model_type': type(model).__name__,
                    'auc_g': np.nan,
                    'auc_p': np.nan,
                    'mce_p': np.nan,
                    'mce_g': np.nan,
                    'ise': np.nan,
                    'classifier': np.nan,
                    'log_loss_list_g': []
                })

        kmm_weights_per_sample = None

        try:
             if len(X_g_test) > 0 and len(X_p_test) > 0 and X_g_test.shape[1] == X_p_test.shape[1]:
                kmm_weights_per_sample = kernel_mean_matching(X_g_test.values, X_p_test.values, kern='rbf', B=1000)

                for info in model_info:
                    kmm_estimate_for_model = np.nan
                    if len(info['log_loss_list_g']) > 0 and len(info['log_loss_list_g']) == len(kmm_weights_per_sample):
                         try:
                             kmm_estimate_for_model = np.mean(info['log_loss_list_g'] * kmm_weights_per_sample)
                         except Exception:
                             pass

                    info['kmm'] = kmm_estimate_for_model

            else:
                for info in model_info:
                     info['kmm'] = np.nan

        except Exception:
            for info in model_info:
                 info['kmm'] = np.nan

        for info in model_info:
             if 'log_loss_list_g' in info:
                  del info['log_loss_list_g']

        model_df = pd.DataFrame(model_info)

        model_df = model_df.dropna(subset=['mce_p']).reset_index(drop=True)

        if len(model_df) == 0:
             final_results[key] = {'Note': f'No models with valid MCE_p found for analysis for {key}'}
             continue

        actual_k = min(K, len(model_df))

        current_dataset_final_results = {}

        oracle_top_k_indices = model_df['mce_p'].nsmallest(actual_k).index
        oracle_selected_models = model_df.loc[oracle_top_k_indices]
        r_final_oracle = oracle_selected_models['mce_p'].mean()
        current_dataset_final_results['Oracle'] = r_final_oracle

        estimation_methods = ['mce_g', 'ise', 'kmm', 'classifier']

        for method in estimation_methods:

            if method not in model_df.columns:
                 current_dataset_final_results[method] = np.nan
                 continue

            estimated_risks_series = model_df[method].dropna()

            if len(estimated_risks_series) == 0:
                 current_dataset_final_results[method] = np.nan
                 continue

            top_k_indices_method = estimated_risks_series.nsmallest(actual_k).index
            selected_models_method = model_df.loc[top_k_indices_method]

            r_final_method = selected_models_method['mce_p'].mean()
            current_dataset_final_results[method] = r_final_method

        final_results[key] = current_dataset_final_results

print_final_header("FINAL RESULTS SUMMARY (Mean R_final for Top-K Models)")

if not final_results:
     print(f"{Fore.RED}No datasets processed or no valid models found in any dataset.{Style.RESET_ALL}")
else:
    for dataset in sorted(final_results.keys()):
        results = final_results[dataset]
        print_final_dataset_header(dataset)

        if 'Note' in results:
             print_final_result("Note", results['Note'], color=Fore.RED)
             continue

        if 'Oracle' in results:
             print_final_result("Oracle (True)", results['Oracle'], color=Fore.GREEN)
        else:
             print_final_result("Oracle (True)", "N/A", color=Fore.RED)

        estimation_methods_to_print = ['mce_g', 'ise', 'kmm', 'classifier']
        method_labels = {'mce_g': 'MCE_G (Empirical)', 'ise': 'ISE (KDE)', 'kmm': 'KMM (Placeholder)', 'classifier': 'Classifier (Domain)'}

        for method_key in estimation_methods_to_print:
            label = method_labels.get(method_key, method_key.upper())
            if method_key in results:
                value = results[method_key]
                if np.isnan(value):
                    print_final_result(label, "NaN", color=Fore.RED, precision=3)
                else:
                    print_final_result(label, value, color=Fore.WHITE)
            else:
                 print_final_result(label, "N/A", color=Fore.RED)

# Species dataset

In [ ]:
datasets_dict = {
     'anemone': pd.read_csv("../datasets/species_new/anemone.csv"),
    'caltha': pd.read_csv("../datasets/species_new/caltha.csv"), 
   # 'hepatica': pd.read_csv("../datasets/species_new/hepatica.csv"),
  #  'convallaria': pd.read_csv("../datasets/species_new/convallaria.csv"),
    #'oxalis': pd.read_csv("../datasets/species_new/oxalis.csv"),
 #  'prunus': pd.read_csv("../datasets/species_new/prunus.csv"),
    'tussilago': pd.read_csv("../datasets/species_new/tussilago.csv"), 
  #  'vaccinium': pd.read_csv("../datasets/species_new/vaccinium.csv")
}

In [ ]:
pd.set_option('future.no_silent_downcasting', True)

do_resampling = True  
do_pca = True

n_components = 2

samples_dict = {}

for key in datasets_dict:
    print(f"\n----- Processing {key} dataset -----")
    df = datasets_dict[key]
    df = df.rename(columns={
        'lat': 'decimalLongitude',
        'long': 'decimalLatitude'
    })
    merged_df = df.dropna()
    print(df["presence"].unique())
    df_list_p = []
    df_list_g = []
    start_year = 2010
    end_year = 2024
    
    for i in range(start_year, end_year + 1):
        set_g = list(range(start_year, i))
        
        set_p = list(range(i, end_year + 1))

        years_g = set_g
        years_p = set_p
        for a in range(1):
            for b in range(1):
                for c in range(1):
                    for d in range(1):
                        if years_g and years_p:
                            df_slice_g = merged_df[merged_df['year'].isin(years_g)]
                            df_slice_p = merged_df[merged_df['year'].isin(years_p)]
                            
                            g_class_counts = df_slice_g['presence'].value_counts()
                            p_class_counts = df_slice_p['presence'].value_counts()
                            
                            def balance_data(df_slice):
                                X = df_slice.drop(['presence', 'year'], axis=1)
                                y = df_slice['presence']
                                
                                if do_resampling and len(y.unique()) > 1 and min(y.value_counts()) < max(y.value_counts()) * 0.8:
                                    smt = SMOTETomek(random_state=42)
                                    X_res, y_res = smt.fit_resample(X, y)
                                    balanced = pd.DataFrame(X_res, columns=X.columns)
                                    balanced['presence'] = y_res
                                    return balanced

                                balanced = pd.DataFrame(X, columns=X.columns)
                                balanced['presence'] = y
                                return balanced
                    
                            balanced_g = balance_data(df_slice_g)
                            balanced_p = balance_data(df_slice_p)
                            
                            target_size = max(len(balanced_g), len(balanced_p))
                            balanced_g = balanced_g.sample(n=target_size, replace=True, random_state=42) if len(balanced_g) < target_size else balanced_g.sample(n=target_size, random_state=42)
                            balanced_p = balanced_p.sample(n=target_size, replace=True, random_state=42) if len(balanced_p) < target_size else balanced_p.sample(n=target_size, random_state=42)
                            
                            balanced_g = balanced_g.rename(columns={
                                'decimalLatitude': 'Latitude',
                                'decimalLongitude': 'Longitude',
                                'presence': 'Metrics'
                            })
                            balanced_p = balanced_p.rename(columns={
                                'decimalLatitude': 'Latitude',
                                'decimalLongitude': 'Longitude',
                                'presence': 'Metrics'
                            })
                            
                            X_source = balanced_g.drop(['Metrics'], axis=1)
                            y_source = balanced_g['Metrics']
                            X_target = balanced_p.drop(['Metrics'], axis=1)
                            y_target = balanced_p['Metrics']
                            
                            X_source_train, X_source_test, y_source_train, y_source_test = train_test_split(X_source, y_source, test_size=0.3, random_state=42)
                            
                            X_target_train, X_target_test, y_target_train, y_target_test = train_test_split(X_target, y_target, test_size=0.3, random_state=42)
                            
                            if do_pca:  
                                
                                pca_source = PCA(n_components=n_components)  
                                X_source_train_pca = pca_source.fit_transform(X_source_train)
                                X_source_test_pca = pca_source.transform(X_source_test)
                                
                                pca_target = PCA(n_components=n_components) 
                                X_target_train_pca = pca_target.fit_transform(X_target_train)
                                X_target_test_pca = pca_target.transform(X_target_test)
                                
                                
                                scaler_source = StandardScaler()
                                X_source_train_scaled = scaler_source.fit_transform(X_source_train_pca)
                                X_source_test_scaled = scaler_source.transform(X_source_test_pca)
                                
                                scaler_target = StandardScaler()
                                X_target_train_scaled = scaler_target.fit_transform(X_target_train_pca)
                                X_target_test_scaled = scaler_target.transform(X_target_test_pca)
                        
                                
                                train_final_source = pd.DataFrame(X_source_train_scaled, 
                                                                columns=[f'PC{i+1}' for i in range(n_components)])
                                test_final_source = pd.DataFrame(X_source_test_scaled, 
                                                               columns=[f'PC{i+1}' for i in range(n_components)])
                                
                                train_final_target = pd.DataFrame(X_target_train_scaled, 
                                                                columns=[f'PC{i+1}' for i in range(n_components)])
                                test_final_target = pd.DataFrame(X_target_test_scaled, 
                                                               columns=[f'PC{i+1}' for i in range(n_components)])
                            else:  
                                
                                scaler_source = StandardScaler()
                                X_source_train_scaled = scaler_source.fit_transform(X_source_train)
                                X_source_test_scaled = scaler_source.transform(X_source_test)
                                
                                scaler_target = StandardScaler()
                                X_target_train_scaled = scaler_target.fit_transform(X_target_train)
                                X_target_test_scaled = scaler_target.transform(X_target_test)
                                
                                
                                feature_names = X_source.columns.tolist()
                                
                                
                                train_final_source = pd.DataFrame(X_source_train_scaled, columns=feature_names)
                                test_final_source = pd.DataFrame(X_source_test_scaled, columns=feature_names)
                                
                                train_final_target = pd.DataFrame(X_target_train_scaled, columns=feature_names)
                                test_final_target = pd.DataFrame(X_target_test_scaled, columns=feature_names)
                        
                            
                            train_final_source['Metrics'] = y_source_train.values
                            test_final_source['Metrics'] = y_source_test.values
                            train_final_target['Metrics'] = y_target_train.values
                            test_final_target['Metrics'] = y_target_test.values
                            
                            df_list_g.append((train_final_source, test_final_source, years_g))
                            df_list_p.append((train_final_target, test_final_target, years_p))
                            
                        samples_dict[key] = {
                            'df_list_p': df_list_p,
                            'df_list_g': df_list_g,
                        }
for key in samples_dict:
    print(f"\nDataset: {key}")
    for i, (train, test, _) in enumerate(samples_dict[key]['df_list_g']):
        print(f"Split {i+1}:")
        print(f"Train shape: {train.shape}")
        print(f"Test shape: {test.shape}")
        print("Train class distribution:", Counter(train['Metrics']))
        print("Test class distribution:", Counter(test['Metrics']))

In [ ]:
all_samples_data = {}

for key in samples_dict:
    all_samples_data[key] = []
    df_list_p = samples_dict[key]['df_list_p']
    df_list_g = samples_dict[key]['df_list_g']
    
    auc_p_values = []
    auc_g_values = []
    
    print(f"Processing: {key}")
    for df_tuple_p, df_tuple_g in zip(df_list_p, df_list_g):
        
        df_p_train, df_p_test, years_p = df_tuple_p
        df_g_train, df_g_test, years_g = df_tuple_g
        
        
        df_p = pd.concat([df_p_train, df_p_test])
        df_g = pd.concat([df_g_train, df_g_test])
        
        
        percent_p = len(df_p[df_p['Metrics'] == 1])/len(df_p)*100
        percent_g = len(df_g[df_g['Metrics'] == 1])/len(df_g)*100
        
        
        sample_p = df_p.drop(columns=['Metrics']).values
        sample_g = df_g.drop(columns=['Metrics']).values

        #similarly edit radius based on dimension and task size, results are sensetive to it!
        rmax = 0.188
        rmin = rmax/10
        r_values = np.arange(0, rmax, rmax/50)
        
        AUC_p = LCF_AUC_from_sample(sample_p, key, "source", r_values, rmin, rmax, False)
        AUC_g = LCF_AUC_from_sample(sample_g, key, "target", r_values, rmin, rmax, False)
        
        all_samples_data[key].append({
            'p_data': (df_p_train, df_p_test, years_p),
            'g_data': (df_g_train, df_g_test, years_g),
            'AUC_p': AUC_p,
            'AUC_g': AUC_g,
            'percent_p': percent_p,
            'percent_g': percent_g
        })
        
        auc_p_values.append(AUC_p)
        auc_g_values.append(AUC_g)
        
        print(f"AUC_p: {AUC_p:.3f}, AUC_g: {AUC_g:.3f}")
        print(f"Class balance - P: {percent_p:.1f}%, G: {percent_g:.1f}%\n")
    
    plt.figure(figsize=(8, 6))
    sns.boxplot(data=[auc_p_values, auc_g_values])
    plt.xticks([0, 1], ['AUC_p', 'AUC_g'])
    plt.title(f'AUC Distribution for {key}')
    plt.legend()
    plt.show()

In [ ]:
def filter_samples(AUC_p_thr=0.15, AUC_g_thr=0.15):
    samples_dict_filtered = {}
    
    for key in all_samples_data:
        samples_dict_filtered[key] = {'df_list_p': [], 'df_list_g': []}
        print(f"\nFiltering: {key}")
        
        for sample in all_samples_data[key]:
            AUC_p = sample['AUC_p']
            AUC_g = sample['AUC_g']
            percent_p = sample['percent_p']
            percent_g = sample['percent_g']
            
            print(f"Checking sample - AUC_p: {AUC_p:.3f}, AUC_g: {AUC_g:.3f}")
            
            if (AUC_p < AUC_p_thr and 
                abs(AUC_g) > AUC_g_thr and 
                percent_p < 100 and 
                percent_g < 100 and AUC_p > 0):
                
                print("--> Passed filter")
                samples_dict_filtered[key]['df_list_p'].append(
                    (*sample['p_data'], "source", AUC_p)
                )
                samples_dict_filtered[key]['df_list_g'].append(
                    (*sample['g_data'], "target", AUC_g)
                )
        
        print(f"{key}")
        print(f"Retained {len(samples_dict_filtered[key]['df_list_p'])} p-samples")
        print(f"Retained {len(samples_dict_filtered[key]['df_list_g'])} g-samples")
    
    return samples_dict_filtered

samples_dict_filtered = filter_samples(AUC_p_thr=0.15, AUC_g_thr=0.15)

In [ ]:
base_output_dir = './pca' 
os.makedirs(base_output_dir, exist_ok=True)

def create_filename_from_years(years_meta_list, default_name="unknown_period"):
    if isinstance(years_meta_list, (list, tuple)) and len(years_meta_list) >= 1:
        if len(years_meta_list) == 1:
            return str(years_meta_list[0])
        return f"{years_meta_list[0]}-{years_meta_list[-1]}"
    return default_name

for key in samples_dict_filtered:
    print(f"\nProcessing cell combination: {key}")

    if 'df_list_p' in samples_dict_filtered[key] and samples_dict_filtered[key]['df_list_p']:
        p_output_dir = os.path.join(base_output_dir, 'p', key)
        os.makedirs(p_output_dir, exist_ok=True)
        print(f"  Saving P-type samples to: {p_output_dir}")

        for i, df_tuple_p in enumerate(samples_dict_filtered[key]['df_list_p']):
            try:
                df_p_train, df_p_test, years_p_meta, p_sample_label, auc_p = df_tuple_p
            except ValueError:
                print(f"    Warning: Could not unpack P-type tuple #{i} for {key}. Skipping.")
                continue

            df_p = pd.concat([df_p_train, df_p_test], ignore_index=True)

            if 'Metrics' not in df_p.columns:
                print(f"    Warning: 'Metrics' column missing in P-sample data for {p_sample_label} ({key}). Skipping.")
                continue
            
            df_p_features = df_p.drop(columns=['Metrics'], errors='ignore')

            if df_p_features.shape[1] == 2:
                feature_cols = df_p_features.columns.tolist()
                filename_base = create_filename_from_years(years_p_meta, default_name=f"p_sample_{i}")
                
       
                csv_path = os.path.join(p_output_dir, f"{filename_base}.csv")
                try:
                    df_p[feature_cols + ['Metrics']].to_csv(csv_path, index=False)
                except Exception as e:
                    print(f"    Error saving CSV for P-sample {filename_base} ({key}): {e}")
                    continue

              
                plt.figure(figsize=(8, 6))
                try:
                    scatter = plt.scatter(
                        df_p[feature_cols[0]], df_p[feature_cols[1]],
                        c=df_p['Metrics'], cmap='coolwarm', alpha=0.6, edgecolors='k', linewidth=0.5
                    )
                    plt.title(f"P-type: {key} - {p_sample_label} ({filename_base})\nAUC: {auc_p:.3f}", fontsize=10)
                    plt.xlabel(feature_cols[0])
                    plt.ylabel(feature_cols[1])
                    if len(df_p['Metrics'].unique()) > 1 : 
                         plt.legend(*scatter.legend_elements(), title="Classes")
                    plt.grid(True, linestyle='--', alpha=0.7)
                    
                    plot_path = os.path.join(p_output_dir, f"{filename_base}.png")
                    plt.savefig(plot_path)
                except Exception as e:
                    print(f"    Error plotting/saving PNG for P-sample {filename_base} ({key}): {e}")
                finally:
                    plt.close() 
            else:
                print(f"    Skipping P-sample {p_sample_label} ({key}): Not 2 features (found {df_p_features.shape[1]}) for plotting.")
    else:
        print(f"  No P-type samples found for {key} or list is empty.")

    
    if 'df_list_g' in samples_dict_filtered[key] and samples_dict_filtered[key]['df_list_g']:
        g_output_dir = os.path.join(base_output_dir, 'g', key)
        os.makedirs(g_output_dir, exist_ok=True)
        print(f"  Saving G-type samples to: {g_output_dir}")

        for i, df_tuple_g in enumerate(samples_dict_filtered[key]['df_list_g']):
            try:
    
                df_g_train, df_g_test, years_g_meta, g_sample_label, auc_g = df_tuple_g
            except ValueError:
                print(f"    Warning: Could not unpack G-type tuple #{i} for {key}. Skipping.")
                continue

            df_g = pd.concat([df_g_train, df_g_test], ignore_index=True)

            if 'Metrics' not in df_g.columns:
                print(f"    Warning: 'Metrics' column missing in G-sample data for {g_sample_label} ({key}). Skipping.")
                continue
            
            df_g_features = df_g.drop(columns=['Metrics'], errors='ignore')

            if df_g_features.shape[1] == 2:
                feature_cols = df_g_features.columns.tolist()
                filename_base = create_filename_from_years(years_g_meta, default_name=f"g_sample_{i}")

               
                csv_path = os.path.join(g_output_dir, f"{filename_base}.csv")
                try:
                    df_g[feature_cols + ['Metrics']].to_csv(csv_path, index=False)
                except Exception as e:
                    print(f"    Error saving CSV for G-sample {filename_base} ({key}): {e}")
                    continue
                
                
                plt.figure(figsize=(8, 6))
                try:
                    scatter = plt.scatter(
                        df_g[feature_cols[0]], df_g[feature_cols[1]],
                        c=df_g['Metrics'], cmap='coolwarm', alpha=0.6, edgecolors='k', linewidth=0.5
                    )
                    plt.title(f"G-type: {key} - {g_sample_label} ({filename_base})\nAUC: {auc_g:.3f}", fontsize=10)
                    plt.xlabel(feature_cols[0])
                    plt.ylabel(feature_cols[1])
                    if len(df_g['Metrics'].unique()) > 1 : 
                        plt.legend(*scatter.legend_elements(), title="Classes")
                    plt.grid(True, linestyle='--', alpha=0.7)

                    plot_path = os.path.join(g_output_dir, f"{filename_base}.png")
                    plt.savefig(plot_path)
                except Exception as e:
                    print(f"    Error plotting/saving PNG for G-sample {filename_base} ({key}): {e}")
                finally:
                    plt.close()
            else:
                print(f"    Skipping G-sample {g_sample_label} ({key}): Not 2 features (found {df_g_features.shape[1]}) for plotting.")
    else:
        print(f"  No G-type samples found for {key} or list is empty.")

print("\nFinished processing all samples.")

In [ ]:
init(autoreset=True)

def custom_log_loss(y_true, y_pred_prob):
    epsilon = 1e-15  # Small constant to avoid log(0)
    y_pred_prob = np.clip(y_pred_prob, epsilon, 1 - epsilon)
    
    log_loss_value = -np.mean(y_true * np.log(y_pred_prob) + (1 - y_true) * np.log(1 - y_pred_prob))
    
    return log_loss_value

def print_header(text):
    """Print a formatted header"""
    print(f"\n{Fore.CYAN}{Style.BRIGHT}{'=' * 80}")
    print(f"{Fore.CYAN}{Style.BRIGHT}  {text}")
    print(f"{Fore.CYAN}{Style.BRIGHT}{'=' * 80}{Style.RESET_ALL}")

def print_subheader(text):
    """Print a formatted subheader"""
    print(f"\n{Fore.GREEN}{Style.BRIGHT}{'-' * 60}")
    print(f"{Fore.GREEN}{Style.BRIGHT}  {text}")
    print(f"{Fore.GREEN}{Style.BRIGHT}{'-' * 60}{Style.RESET_ALL}")

def print_result(label, value, color=Fore.WHITE, precision=3):
    """Print a formatted result"""
    if isinstance(value, (int, float)):
        print(f"{color}{label}: {value:.{precision}f}{Style.RESET_ALL}")
    else:
        print(f"{color}{label}: {value}{Style.RESET_ALL}")

models = {
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=1000, penalty='l2', random_state=42, C=100,),
    "RandomForest": RandomForestClassifier(random_state=42),
        "NeuralNetwork": MLPClassifier(
        hidden_layer_sizes=(8, 4),  
        activation='relu',           
        solver='adam',               
        alpha=0.01,                 
        batch_size='auto',           
        learning_rate='constant',    
        learning_rate_init=10,    
        max_iter=5,                 
        early_stopping=True,         
        validation_fraction=0.1,     
        n_iter_no_change=100        
    )
    
}

all_results = {model_name: {'MCE_g': [], 'MCE_p': [], 'ISE': [], 'KMM': [], 'Classifier': []} 
               for model_name in models.keys()}

for key in samples_dict_filtered:
    print_header(f"Dataset: {key}")
    
    for model_name, model in models.items():
        print_subheader(f"Model: {model_name}")
        results = {'MCE_g': [], 'MCE_p': [], 'ISE': [], 'KMM': [], 'Classifier': []}
        
        for tuple_p, tuple_g in zip(samples_dict_filtered[key]['df_list_p'], samples_dict_filtered[key]['df_list_g']):
            if len(samples_dict_filtered[key]['df_list_p']) > 0: 
                df_p_train, df_p_test, AUC_p, years_p = tuple_p[0], tuple_p[1], tuple_p[3], tuple_p[2]  
                df_g_train, df_g_test, AUC_g, years_g = tuple_g[0], tuple_g[1], tuple_g[3], tuple_g[2]
                
 
                X_p_train = df_p_train.drop(columns=['Metrics'])
                y_p_train = df_p_train['Metrics']
                X_p_test = df_p_test.drop(columns=['Metrics'])
                y_p_test = df_p_test['Metrics']
                
                X_g_train = df_g_train.drop(columns=['Metrics'])
                y_g_train = df_g_train['Metrics']
                X_g_test = df_g_test.drop(columns=['Metrics'])
                y_g_test = df_g_test['Metrics']
                
                clf = model
                start_time = time.time()
                clf.fit(X_g_train, y_g_train)
                training_time = time.time() - start_time
                
                y_pred_prob_g = clf.predict_proba(X_g_test)[:, 1]
                auc_test = roc_auc_score(y_g_test, y_pred_prob_g)
                
                print_result("Sample sizes", f"P train: {len(y_p_train)}, G train: {len(y_g_train)}")
                print_result("AUC values", f"P: {AUC_p}, G: {AUC_g}")
                print_result("Test ROC AUC", auc_test, color=Fore.YELLOW)
                print_result("Training time", f"{training_time:.2f} seconds", color=Fore.CYAN)
                
                if auc_test > 0.7:
                    log_loss_list_g = [custom_log_loss(y_true, y_pred) 
                                       for y_true, y_pred in zip(y_g_test, y_pred_prob_g)]
                    
                    y_pred_prob_p = clf.predict_proba(X_p_test)[:, 1]
                    log_loss_list_p = [custom_log_loss(y_true, y_pred) 
                                       for y_true, y_pred in zip(y_p_test, y_pred_prob_p)]
                    
                    mce_p_mean = np.mean(log_loss_list_p)
                    mce_g_mean = np.mean(log_loss_list_g)
                    
                    kde_scott_g = gaussian_kde(X_g_train.values.T, bw_method='scott')
                    kde_scott_p = gaussian_kde(X_p_train.values.T, bw_method='scott')
                    bw_scott_g = kde_scott_g.factor
                    bw_scott_p = kde_scott_p.factor
                    
                    print_result("Bandwidth values", f"P: {bw_scott_p:.5f}, G: {bw_scott_g:.5f}")
                    
                    is_scott = [error * kde_scott_p(point) / kde_scott_g(point) 
                                for error, point in zip(log_loss_list_g, X_g_test.values)]
                    
                    kmm = kernel_mean_matching(X_p_test.values, X_g_test.values, kern='lin', B=1000)
                    kmm_mean = np.mean([error * w for error, w in zip(log_loss_list_g, kmm)])
                    
                    X_combined = np.concatenate([X_g_train.values, X_p_train.values])
                    y_combined = np.concatenate([np.ones(len(X_g_train)), np.zeros(len(X_p_train))])
                    
                    domain_clf = LogisticRegression(max_iter=10, C = 100)
                    domain_clf.fit(X_combined, y_combined)
                    
                    domain_probs = domain_clf.predict_proba(X_g_test.values)[:, 1]
                    classifier_weights = np.clip((1) / (domain_probs + 1e-10), 1e-7, 2)
                    classifier_mean = np.mean([error * weight for error, weight in zip(log_loss_list_g, classifier_weights)])
                    
                    print_result("MCE P", mce_p_mean, color=Fore.MAGENTA)
                    print_result("MCE G", mce_g_mean, color=Fore.MAGENTA)
                    print_result("ISE", np.mean(is_scott), color=Fore.MAGENTA)
                    print_result("KMM", kmm_mean, color=Fore.MAGENTA)
                    print_result("Classifier", classifier_mean, color=Fore.MAGENTA)
                    print_result("Years", f"{years_p} -> {years_g}", color=Fore.WHITE)
                    
                    results["MCE_g"].append(mce_g_mean)
                    results["MCE_p"].append(mce_p_mean) 
                    results["KMM"].append(kmm_mean) 
                    results["ISE"].append(np.mean(is_scott))
                    results["Classifier"].append(classifier_mean)
                    
                else:
                    print_result(f"Low ROC AUC for {key}", auc_test, color=Fore.RED)
                
                print(f"{Fore.WHITE}{'-' * 40}{Style.RESET_ALL}")
        
        for method in results:
            all_results[model_name][method].extend(results[method])
        
        if results["MCE_p"]:  # Only if we have results
            mean_metrics_results = {'MCE_g': None, 'ISE': None, 'KMM': None, 'Classifier': None}
            print_subheader(f"Metrics Summary for {model_name} on {key}")
            
            for method, values_list in results.items():
                if method != "MCE_p":
                    actual = np.array(results["MCE_p"])
                    predicted = np.array(values_list)
                    epsilon = 1e-10
                    actual_adj = np.maximum(actual, epsilon)
                    
                    mape = np.mean(np.abs((actual - predicted) / actual_adj)) * 100
                    rmse = np.sqrt(np.mean((actual - predicted) ** 2))
                    rmspe = np.sqrt(np.mean(((actual - predicted) / actual_adj) ** 2)) * 100
                    
                    mean_metrics_results[method] = {
                        'MAPE': mape,
                        'RMSE': rmse,
                        'RMSPE': rmspe
                    }
            
            for metric_name in ['MAPE', 'RMSE', 'RMSPE']:
                print(f"\n{Fore.BLUE}{Style.BRIGHT}{metric_name}:{Style.RESET_ALL}")
                for method in ['MCE_g', 'ISE', 'KMM', 'Classifier']:
                    metric_value = mean_metrics_results[method][metric_name]
                    print_result(f"  {method}", metric_value)

print_header("OVERALL RESULTS")

for model_name in models:
    model_results = all_results[model_name]
    if model_results["MCE_p"]:  # Only if we have results
        mean_metrics_results_all = {'MCE_g': None, 'ISE': None, 'KMM': None, 'Classifier': None}
        print_subheader(f"Overall Metrics for {model_name}")
        
        for method, values_list in model_results.items():
            if method != "MCE_p":
                actual = np.array(model_results["MCE_p"])
                predicted = np.array(values_list)
                epsilon = 1e-10
                actual_adj = np.maximum(actual, epsilon)
                
                mape = np.mean(np.abs((actual - predicted) / actual_adj)) * 100
                rmse = np.sqrt(np.mean((actual - predicted) ** 2))
                rmspe = np.sqrt(np.mean(((actual - predicted) / actual_adj) ** 2)) * 100
                
                mean_metrics_results_all[method] = {
                    'MAPE': mape,
                    'RMSE': rmse,
                    'RMSPE': rmspe
                }
        
        for metric_name in ['MAPE', 'RMSE', 'RMSPE']:
            print(f"\n{Fore.BLUE}{Style.BRIGHT}{metric_name}:{Style.RESET_ALL}")
            for method in ['MCE_g', 'ISE', 'KMM', 'Classifier']:
                metric_value = mean_metrics_results_all[method][metric_name]
                print_result(f"  {method}", metric_value)

In [ ]:
def custom_log_loss(y_true, y_pred_prob):
    epsilon = 1e-15
    y_pred_prob = np.clip(y_pred_prob, epsilon, 1 - epsilon)
    log_loss_value = -np.mean(y_true * np.log(y_pred_prob) + (1 - y_true) * np.log(1 - y_pred_prob))
    return log_loss_value

def print_header(text):
    print(f"\n{Fore.CYAN}{Style.BRIGHT}{'=' * 80}")
    print(f"{Fore.CYAN}{Style.BRIGHT}  {text}")
    print(f"{Fore.CYAN}{Style.BRIGHT}{'=' * 80}{Style.RESET_ALL}")

def print_result(label, value, color=Fore.WHITE, precision=3):
    if isinstance(value, (int, float)):
        print(f"{color}{label}: {value:.{precision}f}{Style.RESET_ALL}")
    else:
        print(f"{color}{label}: {value}{Style.RESET_ALL}")

def generate_models(n_models=2000):
    models = []
    for i in range(n_models):
        if i < n_models // 2:
            model = GradientBoostingClassifier(
                random_state=i,
                n_estimators=np.random.randint(5, 20),
                max_depth=np.random.randint(1, 4),
                learning_rate=np.random.uniform(0.01, 0.3),
                subsample=np.random.uniform(0.3, 0.8)
            )
        else:
            model = RandomForestClassifier(
                random_state=i,
                n_estimators=np.random.randint(5, 20),
                max_depth=np.random.randint(1, 4),
                max_features=np.random.uniform(0.3, 0.8),
                min_samples_split=np.random.randint(5, 20)
            )
        models.append(model)
    return models

K = 5
n_models = 2000

final_results = {}

for key in samples_dict_filtered:
    print_header(f"Dataset: {key}")
    
    if len(samples_dict_filtered[key]['df_list_p']) > 0:
        tuple_p = samples_dict_filtered[key]['df_list_p'][0]
        tuple_g = samples_dict_filtered[key]['df_list_g'][0]
        
        df_p_train, df_p_test, AUC_p, years_p = tuple_p[0], tuple_p[1], tuple_p[3], tuple_p[2]
        df_g_train, df_g_test, AUC_g, years_g = tuple_g[0], tuple_g[1], tuple_g[3], tuple_g[2]
        
        X_p_train = df_p_train.drop(columns=['Metrics'])
        y_p_train = df_p_train['Metrics']
        X_p_test = df_p_test.drop(columns=['Metrics'])
        y_p_test = df_p_test['Metrics']
        
        X_g_train = df_g_train.drop(columns=['Metrics'])
        y_g_train = df_g_train['Metrics']
        X_g_test = df_g_test.drop(columns=['Metrics'])
        y_g_test = df_g_test['Metrics']
        
        models = generate_models(n_models)
        
        all_risks = {
            'MCE_p': [],
            'MCE_g': [],
            'ISE': [],
            'KMM': [],
            'Classifier': []
        }
        
        kde_scott_g = gaussian_kde(X_g_train.values.T, bw_method='scott')
        kde_scott_p = gaussian_kde(X_p_train.values.T, bw_method='scott')
        
        X_combined = np.concatenate([X_g_train.values, X_p_train.values])
        y_combined = np.concatenate([np.ones(len(X_g_train)), np.zeros(len(X_p_train))])
        domain_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)

        domain_clf.fit(X_combined, y_combined)
        
        model_info = []
        
        for i, model in enumerate(models):
            if i % 50 == 0:
                print(f"Processing model {i}/{n_models}")
            
            model.fit(X_g_train, y_g_train)
            
            y_pred_prob_g = model.predict_proba(X_g_test)[:, 1]
            y_pred_prob_p = model.predict_proba(X_p_test)[:, 1]
            
            auc_g = roc_auc_score(y_g_test, y_pred_prob_g)
            auc_p = roc_auc_score(y_p_test, y_pred_prob_p)
            
            log_loss_list_g = [custom_log_loss(y_true, y_pred) 
                              for y_true, y_pred in zip(y_g_test, y_pred_prob_g)]
            log_loss_list_p = [custom_log_loss(y_true, y_pred) 
                              for y_true, y_pred in zip(y_p_test, y_pred_prob_p)]
            
            mce_p_mean = np.mean(log_loss_list_p)
            mce_g_mean = np.mean(log_loss_list_g)
            
            is_scott = [error * kde_scott_p(point) / kde_scott_g(point) 
                       for error, point in zip(log_loss_list_g, X_g_test.values)]
            
            kmm = kernel_mean_matching(X_p_test.values, X_g_test.values, kern='rbf', B=1000)
            # print(kmm)
            kmm_mean = np.mean([error * w for error, w in zip(log_loss_list_g, kmm)])
            
            domain_probs = domain_clf.predict_proba(X_g_test.values)[:, 1]
            classifier_weights = np.clip((1) / (domain_probs), 1e-15, 2000)
            # print(classifier_weights)
            classifier_mean = np.mean([error * weight for error, weight in zip(log_loss_list_g, classifier_weights)])
            
            all_risks['MCE_p'].append(mce_p_mean)
            all_risks['MCE_g'].append(mce_g_mean)
            all_risks['ISE'].append(np.mean(is_scott))
            all_risks['KMM'].append(kmm_mean)
            all_risks['Classifier'].append(classifier_mean)
            
            model_info.append({
                'model_id': i,
                'model_type': type(model).__name__,
                'auc_g': auc_g,
                'auc_p': auc_p,
                'mce_p': mce_p_mean,
                'mce_g': mce_g_mean,
                'ise': np.mean(is_scott),
                'kmm': kmm_mean,
                'classifier': classifier_mean,
                'kde_p_factor': kde_scott_p.factor,
                'kde_g_factor': kde_scott_g.factor,
                'domain_prob_mean': np.mean(domain_probs),
                'domain_prob_std': np.std(domain_probs),
                'classifier_weights_mean': np.mean(classifier_weights),
                'classifier_weights_std': np.std(classifier_weights),
                'kmm_weights_mean': np.mean(kmm),
                'kmm_weights_std': np.std(kmm),
                'ise_weights_mean': np.mean([kde_scott_p(point) / kde_scott_g(point) for point in X_g_test.values]),
                'ise_weights_std': np.std([kde_scott_p(point) / kde_scott_g(point) for point in X_g_test.values])
            })
        
        print_header(f"Model Statistics for {key}")
        model_df = pd.DataFrame(model_info)
        
        print_result("AUC G - Mean", model_df['auc_g'].mean(), color=Fore.BLUE)
        print_result("AUC G - Std", model_df['auc_g'].std(), color=Fore.BLUE)
        print_result("AUC G - Min", model_df['auc_g'].min(), color=Fore.BLUE)
        print_result("AUC G - Max", model_df['auc_g'].max(), color=Fore.BLUE)
        
        print_result("AUC P - Mean", model_df['auc_p'].mean(), color=Fore.BLUE)
        print_result("AUC P - Std", model_df['auc_p'].std(), color=Fore.BLUE)
        print_result("AUC P - Min", model_df['auc_p'].min(), color=Fore.BLUE)
        print_result("AUC P - Max", model_df['auc_p'].max(), color=Fore.BLUE)
        
        print_result("Domain Prob Mean", model_df['domain_prob_mean'].mean(), color=Fore.CYAN)
        print_result("Domain Prob Std", model_df['domain_prob_std'].mean(), color=Fore.CYAN)
        
        print_result("Classifier Weights Mean", model_df['classifier_weights_mean'].mean(), color=Fore.CYAN)
        print_result("Classifier Weights Std", model_df['classifier_weights_std'].mean(), color=Fore.CYAN)
        
        print_result("KMM Weights Mean", model_df['kmm_weights_mean'].mean(), color=Fore.CYAN)
        print_result("KMM Weights Std", model_df['kmm_weights_std'].mean(), color=Fore.CYAN)
        
        print_result("ISE Weights Mean", model_df['ise_weights_mean'].mean(), color=Fore.CYAN)
        print_result("ISE Weights Std", model_df['ise_weights_std'].mean(), color=Fore.CYAN)
        
        print_result("KDE P Factor", model_df['kde_p_factor'].iloc[0], color=Fore.YELLOW)
        print_result("KDE G Factor", model_df['kde_g_factor'].iloc[0], color=Fore.YELLOW)
        
        print_header(f"Risk Method Correlation Analysis for {key}")
        risk_corr = model_df[['mce_p', 'mce_g', 'ise', 'kmm', 'classifier']].corr()
        print("Correlation Matrix:")
        print(risk_corr.round(3))
        
        print_header(f"Top Models Analysis for {key}")
        
        final_results[key] = {}
        
        for method in ['MCE_g', 'ISE', 'KMM', 'Classifier']:
            estimated_risks = np.array(all_risks[method])
            true_risks = np.array(all_risks['MCE_p'])
            
            top_k_indices = np.argsort(estimated_risks)[:K]
            selected_models = model_df.iloc[top_k_indices]
            
            r_final_method = np.mean(true_risks[top_k_indices])
            final_results[key][method] = r_final_method
            
            print(f"\n{Fore.GREEN}Method: {method}{Style.RESET_ALL}")
            print_result(f"R_final_{method}", r_final_method, color=Fore.MAGENTA)
            print_result("Selected Model AUCs G", f"{selected_models['auc_g'].mean():.3f} ± {selected_models['auc_g'].std():.3f}")
            print_result("Selected Model AUCs P", f"{selected_models['auc_p'].mean():.3f} ± {selected_models['auc_p'].std():.3f}")
            print_result("Selected Models Types", f"GB: {sum(selected_models['model_type'] == 'GradientBoostingClassifier')}, RF: {sum(selected_models['model_type'] == 'RandomForestClassifier')}")
        
        baseline_top_k = np.argsort(all_risks['MCE_p'])[:K]
        oracle_risk = np.mean(np.array(all_risks['MCE_p'])[baseline_top_k])
        oracle_models = model_df.iloc[baseline_top_k]
        final_results[key]['Oracle'] = oracle_risk
        
        print(f"\n{Fore.GREEN}Method: Oracle{Style.RESET_ALL}")
        print_result("R_final_Oracle", oracle_risk, color=Fore.GREEN)
        print_result("Oracle Model AUCs G", f"{oracle_models['auc_g'].mean():.3f} ± {oracle_models['auc_g'].std():.3f}")
        print_result("Oracle Model AUCs P", f"{oracle_models['auc_p'].mean():.3f} ± {oracle_models['auc_p'].std():.3f}")
        print_result("Oracle Models Types", f"GB: {sum(oracle_models['model_type'] == 'GradientBoostingClassifier')}, RF: {sum(oracle_models['model_type'] == 'RandomForestClassifier')}")
        
        final_results[key]['model_stats'] = {
            'total_models': len(models),
            'auc_g_range': [model_df['auc_g'].min(), model_df['auc_g'].max()],
            'auc_p_range': [model_df['auc_p'].min(), model_df['auc_p'].max()],
            'risk_correlations': risk_corr['mce_p'].to_dict()
        }

print_header("FINAL RESULTS SUMMARY")
for dataset, results in final_results.items():
    print(f"\n{Fore.YELLOW}{Style.BRIGHT}Dataset: {dataset}{Style.RESET_ALL}")
    for method, risk in results.items():
        color = Fore.GREEN if method == 'Oracle' else Fore.WHITE
        print_result(f"  {method}", risk, color=color)